In [15]:
# import dictionaries
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt

In [16]:
# here we load in our dictionary of player's serve and return statistics, scraped from atpworldtour.com
import cPickle as pickle
with open('stats_dict.p', 'rb') as fp:
    stats_dict = pickle.load(fp)

In [17]:
# function to convert dict of two-element vector scores into proper overall score; does not check for valid scores
def get_score(points):
    # unless we are in a tiebreaker, code converts point score from number format to tennis' 15-30-40-deuce format
    if points['game'] != [6,6]:
        point_score = [int(p)*15 for p in points['point']]
        for i in range(0,len(point_score)):
            if point_score[i] == 45:
                point_score[i] = 40
        if max(point_score) > 40:
            if point_score[0] > point_score[1]:
                point_score = ['AD','40']
            if point_score[0] < point_score[1]:
                point_score = ['40','AD']
            if point_score[0] == point_score[1]:
                point_score = ['40','40']
        return str(points['set'][0]) + '-' + str(points['set'][1]) + ', ' + str(points['game'][0]) +\
        '-' + str(points['game'][1]) + ', ' + str(point_score[0]) + '-' + str(point_score[1])
    else:
        return str(points['set'][0]) + '-' + str(points['set'][1]) + ', ' + str(points['game'][0]) +\
        '-' + str(points['game'][1]) + ', ' + str(points['point'][0]) + '-' + str(points['point'][1])


In [18]:
# check to see if a game should be over in tennis
# I will first write this treating the tiebreaker as a typical fifteenth game with a single server
# then I will later update it to reflect an actual tiebreaker
score = {}
score['set'] = [1,0]
score['game'] = [6,6]
score['point'] = [7,9]

def check_score(points):
    # first we check the points score    
    # if we are not in a tiebreaker, proceed with regular scoring
    if points['game'] != [6,6]:
        if max(points['point']) < 4:
            return points
        if max(points['point']) >= 4:
            if (max(points['point'])-min(points['point'])) < 2:
                return points
            else:
                # if the difference in points is 2, we reset the point score and update the game score
                if points['point'][0] > points['point'][1]:
                    points['game'][0] += 1
                    # I switch the server after any time the game score is updated
                    index = server_I.next()
                    global server
                    global serving
                    server = serving[index]
                if points['point'][1] > points['point'][0]:
                    points['game'][1] += 1
                    index = server_I.next()
                    global server
                    global serving
                    server = serving[index]
                points['point'] = [0,0]
    #if we are in a tiebreaker, switch serve every two points and end once a player gets to seven and wins by two
    elif points['game'] == [6,6]:
        # now check if tiebreak should be over
        if max(points['point']) >= 7 and ((max(points['point'])-min(points['point'])) >= 2):
            if points['point'][0] > points['point'][1]:
                points['game'][0] += 1
                # I switch the server after any time the game score is updated
                index = server_I.next()
                global server
                global serving
                server = serving[index]
            if points['point'][1] > points['point'][0]:
                points['game'][1] += 1
                index = server_I.next()
                global server
                global serving
                server = serving[index]
            points['point'] = [0,0]
        # otherwise, tiebreak continues
        else:
            return points
            
    # now we check the game score
    if max(points['game']) < 6:
        return points
    if points['game'] == [6,6] and points['point'] == [0,0]:
        return points
    # my next logical statement covers all cases where a player has reached 6 or more games except 6-5
    if (max(points['game']) - min(points['game'])) >= 2 or max(points['game']) == 7:
        if points['game'][0] > points['game'][1]:
            points['set'][0] += 1
        if points['game'][1] > points['game'][0]:
            points['set'][1] += 1
        points['game'] = [0,0]

    # now we check the set score
    if max(points['set']) < 2:
        return points
    else:
        #print "Game, Set, Match."
        return points


<ipython-input-18-fc2e0905781e>:30: SyntaxWarning: name 'server' is assigned to before global declaration
  global server
<ipython-input-18-fc2e0905781e>:31: SyntaxWarning: name 'serving' is used prior to global declaration
  global serving
<ipython-input-18-fc2e0905781e>:42: SyntaxWarning: name 'server' is assigned to before global declaration
  global server
<ipython-input-18-fc2e0905781e>:43: SyntaxWarning: name 'serving' is used prior to global declaration
  global serving
<ipython-input-18-fc2e0905781e>:48: SyntaxWarning: name 'server' is assigned to before global declaration
  global server
<ipython-input-18-fc2e0905781e>:49: SyntaxWarning: name 'serving' is used prior to global declaration
  global serving


In [19]:
def point(first_serve_perc,first_serve_win,second_serve_win,score):
    rando = np.random.uniform()
    if rando < first_serve_perc:
        point = np.random.binomial(1,first_serve_win)
    if rando > first_serve_perc:
        point = np.random.binomial(1,second_serve_win)
    #point_chex.append(point)
    # depending on who is serving, I must assign the won and lost points accordingly
    if (point == 1 and server == 'p1') or (point == 0 and server == 'p2'):
        score[0] += 1
    else:
        score[1] += 1
    return score

In [20]:
# let's get started

def simulate(mast_dict,p1,p2):
    outcomes = []
    
    # set up indicator variable to designate who is serving
    global server_I
    server_I = cycle(range(2))
    # set up dict so I can index my iterator to find the server in a given game
    global serving
    serving = ['p1','p2']
    global server
    
    # flip a coin to get this match started
    if np.random.uniform(0,1) > .5:
        #print "%s to serve" % (p1)
        index = server_I.next()
        server = serving[index]
    else:
        #print "%s to serve" % (p2)
        server_I.next()
        index = server_I.next()
        server = serving[index]

    # initialize empty scoreboards
    score = {}
    score['set'] = [0,0]
    score['game'] = [0,0]
    score['point'] = [0,0]

    # now we get the party started...
    while max(check_score(score)['set']) < 2:
        score['point'] = point(mast_dict[server]['1st_serve%'],mast_dict[server]['1st_serve_win'],mast_dict[server]['2nd_serve_win'],score['point'])
        score = check_score(score)
        # checker
        if score['game'] == [6,6] and sum(score['point'])%2 == 1:
            index = server_I.next()
            global server
            global serving
            server = serving[index]
            
    if max(check_score(score)['set']) == 2:
        global names
        if check_score(score)['set'][0] == 2:
            wins[0] += 1
            #print names['p1'] + ' wins.'
        else:
            wins[1] += 1
            #print names['p2'] + ' wins'

<ipython-input-20-9ddc746b598b>:38: SyntaxWarning: name 'server' is assigned to before global declaration
  global server
<ipython-input-20-9ddc746b598b>:39: SyntaxWarning: name 'serving' is assigned to before global declaration
  global serving


In [21]:
def n_match_simulator(player_1,player_2,year,surface,n):
    # names dict for future reference
    global names 
    names = {'p1': player_1,'p2': player_2}
    global wins
    wins = [0,0]
    
    # split our dictionary into five dataframes for each relevant player statistic
    first_perc = stats_dict['1st-serve'][surface][year]
    first_serve = stats_dict['1st-serve-points-won'][surface][year]
    second_serve = stats_dict['2nd-serve-points-won'][surface][year]
    first_return = stats_dict['1st-serve-return-points-won'][surface][year]
    second_return = stats_dict['2nd-serve-return-points-won'][surface][year]
    
    # extract serve and return percentages for player 1 and player 2 from the database
    player1_info = {'1st_serve%':first_perc.loc[first_perc['player'] == player_1]['1st-serve'].values[0],\
                '1st_serve_win':first_serve.loc[first_serve['player'] == player_1]['1st-serve-points-won'].values[0],\
                '2nd_serve_win':second_serve.loc[second_serve['player'] == player_1]['2nd-serve-points-won'].values[0],\
                '1st_return_win':first_return.loc[first_return['player'] == player_1]['1st-serve-return-points-won'].values[0],\
                '2nd_return_win':second_return.loc[second_return['player'] == player_1]['2nd-serve-return-points-won'].values[0]}

    player2_info = {'1st_serve%':first_perc.loc[first_perc['player'] == player_2]['1st-serve'].values[0],\
                '1st_serve_win':first_serve.loc[first_serve['player'] == player_2]['1st-serve-points-won'].values[0],\
                '2nd_serve_win':second_serve.loc[second_serve['player'] == player_2]['2nd-serve-points-won'].values[0],\
                '1st_return_win':first_return.loc[first_return['player'] == player_2]['1st-serve-return-points-won'].values[0],\
                '2nd_return_win':second_return.loc[second_return['player'] == player_2]['2nd-serve-return-points-won'].values[0]}


    player1_dict = {}
    player2_dict = {}


    # normalize percentages by dividing each by sum of players' two stats (eg 1st serve win pct and 1st serve return pct)
    # the new serve winning percentages reflect a combination of the server and returner's relative effectiveness
    player1_dict['1st_serve_win'] = player1_info['1st_serve_win']/float(player1_info['1st_serve_win']+player2_info['1st_return_win'])
    player2_dict['1st_serve_win'] = player2_info['1st_serve_win']/float(player2_info['1st_serve_win']+player1_info['1st_return_win'])
    player1_dict['2nd_serve_win'] = player1_info['2nd_serve_win']/float(player1_info['2nd_serve_win']+player2_info['2nd_return_win'])
    player2_dict['2nd_serve_win'] = player2_info['2nd_serve_win']/float(player2_info['2nd_serve_win']+player1_info['2nd_return_win'])
    player1_dict['1st_serve%'] = player1_info['1st_serve%']
    player2_dict['1st_serve%'] = player2_info['1st_serve%']


    mast_dict = {}
    mast_dict['p1'] = player1_dict
    mast_dict['p2'] = player2_dict
    
    
    
    for i in range(0,n):
        simulate(mast_dict,player_1,player_2)
    print player_1 + ": " + str(wins[0]) + " wins"
    print player_2 + ": " + str(wins[1]) + " wins"
    return float(wins[0])/1000
#n_match_simulator('pete-sampras','andre-agassi',2000,'grass',1000)

n_match_simulator('rafael-nadal','roger-federer',2008,'clay',1000)

rafael-nadal: 646 wins
roger-federer: 354 wins


0.646

In [33]:
x = .647

In [34]:
plt.hist(wins)

(array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 353. ,  382.4,  411.8,  441.2,  470.6,  500. ,  529.4,  558.8,
         588.2,  617.6,  647. ]),
 <a list of 10 Patch objects>)

In [25]:
# now we'll test these probabilities out at this year's season finale, the 2015 ATP World Tour Finals
# Round Robin results, winner is entered as player one

pairs = [['novak-djokovic','kei-nishikori'], ['roger-federer','tomas-berdych'],\
         ['andy-murray','david-ferrer'], ['rafael-nadal','stan-wawrinka'],\
         ['kei-nishikori','tomas-berdych'], ['roger-federer','novak-djokovic'],\
         ['stan-wawrinka','david-ferrer'], ['rafael-nadal','andy-murray'],\
         ['novak-djokovic','tomas-berdych'], ['roger-federer','kei-nishikori'],\
         ['stan-wawrinka','andy-murray'], ['rafael-nadal','david-ferrer'],\
         ['novak-djokovic','rafael-nadal'], ['roger-federer','stan-wawrinka'],\
         ['novak-djokovic','roger-federer']]

m = []
for i in xrange(len(pairs)):
    m.append(n_match_simulator(pairs[i][0],pairs[i][1], 2015, 'hard',1000))
m

novak-djokovic: 619 wins
kei-nishikori: 381 wins
roger-federer: 675 wins
tomas-berdych: 325 wins
andy-murray: 565 wins
david-ferrer: 435 wins
rafael-nadal: 623 wins
stan-wawrinka: 377 wins
kei-nishikori: 463 wins
tomas-berdych: 537 wins
roger-federer: 602 wins
novak-djokovic: 398 wins
stan-wawrinka: 377 wins
david-ferrer: 623 wins
rafael-nadal: 452 wins
andy-murray: 548 wins
novak-djokovic: 588 wins
tomas-berdych: 412 wins
roger-federer: 717 wins
kei-nishikori: 283 wins
stan-wawrinka: 313 wins
andy-murray: 687 wins
rafael-nadal: 486 wins
david-ferrer: 514 wins
novak-djokovic: 616 wins
rafael-nadal: 384 wins
roger-federer: 757 wins
stan-wawrinka: 243 wins
novak-djokovic: 413 wins
roger-federer: 587 wins


"    \nm1 = n_match_simulator('novak-djokovic','kei-nishikori',2015,'hard',1000)\nm2 = n_match_simulator('roger-federer','tomas-berdych',2015,'hard',1000)\n\nm3 = n_match_simulator('andy-murray','david-ferrer',2015,'hard',1000)\nm4 = n_match_simulator('rafael-nadal','stan-wawrinka',2015,'hard',1000)\n\nm5 = n_match_simulator('kei-nishikori','tomas-berdych',2015,'hard',1000)\nm6 = n_match_simulator('roger-federer','novak-djokovic',2015,'hard',1000)\n\nm7 = n_match_simulator('stan-wawrinka','david-ferrer',2015,'hard',1000)\nm8 = n_match_simulator('rafael-nadal','andy-murray',2015,'hard',1000)\n\nm9 = n_match_simulator('novak-djokovic','tomas-berdych',2015,'hard',1000)\nm10 = n_match_simulator('roger-federer','kei-nishikori',2015,'hard',1000)\n\nm11 = n_match_simulator('stan-wawrinka','andy-murray',2015,'hard',1000)\nm12 = n_match_simulator('rafael-nadal','david-ferrer',2015,'hard',1000)\n\n# semifinal and final results, winner entered as player one\n\nm13 = n_match_simulator('novak-djoko

In [29]:
a = []
a.append(m[i] > .5  for i in range(0,len(m)))

In [30]:
a

[<generator object <genexpr> at 0x108254640>]